In [ ]:
# Importing the necessary libraries
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Define directories for negative (no crack) and positive (crack) images
dataset_dir = '/kaggle/input/concrete-crack-images-for-classification'

# Initialize ImageDataGenerator with rescaling and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values between 0 and 1
    rotation_range=15,  # Random rotation between 0-15 degrees
    width_shift_range=0.1,  # Randomly shift images horizontally (10%)
    height_shift_range=0.1,  # Randomly shift images vertically (10%)
    horizontal_flip=True,  # Flip images horizontally
    validation_split=0.2  # Reserve 20% of data for validation
)

# Train generator: loads batches of images from the dataset directory
train_generator = train_datagen.flow_from_directory(
    dataset_dir,  # Dataset directory
    target_size=(227, 227),  # Resize images to 227x227
    batch_size=32,  # Number of images in each batch
    class_mode='categorical',  # Binary classification (crack vs no crack)
    subset='training'  # 80% for training
)

# Validation generator: loads batches for validation
validation_generator = train_datagen.flow_from_directory(
    dataset_dir,  # Same dataset directory
    target_size=(227, 227),  # Same target size for validation
    batch_size=32,  # Batch size
    class_mode='categorical',  # Binary classification
    subset='validation'  # 20% for validation
)


In [ ]:
# Build the CNN model
model = Sequential()

# 1st Convolutional Layer (Conv2D + MaxPooling)
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(227, 227, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 2nd Convolutional Layer (Conv2D + MaxPooling)
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the layers (convert 2D feature maps to 1D feature vectors)
model.add(Flatten())

# Fully Connected Layer (Dense)
model.add(Dense(64, activation='relu'))

# Output Layer (Dense with softmax activation for binary classification)
model.add(Dense(2, activation='softmax'))

# Compile the model with Adam optimizer and categorical crossentropy loss
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary to verify the architecture
model.summary()


In [ ]:
# Train the model with the train and validation data generators
history = model.fit(
    train_generator,  # Training data loaded in batches
    epochs=10,  # Number of training epochs
    validation_data=validation_generator  # Validation data loaded in batches
)


In [ ]:
# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")
